In [1]:
import tensorflow as tf
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter


# Make 2 data points in NumPy.
x_data = np.float32([[1,0]])
y_data = np.float32([0,1])

# Construct a linear model.
b = tf.Variable(np.float32([-1]))
W = tf.Variable(np.float32([[4]]))

y = tf.sigmoid(tf.matmul(W, x_data) + b)

# Minimize the squared errors.
loss = tf.reduce_mean(tf.square(y - y_data))
optimizer = tf.train.GradientDescentOptimizer(50.0)
train = optimizer.minimize(loss)

# For initializing the variables.
init = tf.global_variables_initializer()

# Launch the graph
sess = tf.Session()
sess.run(init)
print (sess.run(W), sess.run(b), sess.run(loss))

W_grid = []
b_grid = []
loss_grid = []

# Training
for step in range(1, 13):
    sess.run(train)
    if step % 1 == 0:
        W_grid.append(sess.run(W[0][0]))
        b_grid.append(sess.run(b[0]))
        loss_grid.append(sess.run(loss))
        print(step, sess.run(W), sess.run(b), sess.run(loss))
        
result = sess.run(y)[0]

for i, value in enumerate(x_data[0]):
    print('Tensor raw input: ', value)
    print('Tensor inverted ouput: ', result[i])
    
    if round(value) == (not round(result[i])):
        print('Tensor aproximately calculate NOT function')
    else:
        print('Tensor does not calculate NOT function')


[[4.]] [-1.] 0.7209221
1 [[1.8482943]] [4.0350366] 0.49737713
2 [[1.7101799]] [3.9117591] 0.49659425
3 [[1.5312474]] [3.7516825] 0.49522403
4 [[1.2812005]] [3.5273435] 0.49234644
5 [[0.8830391]] [3.1687577] 0.48386672
6 [[0.05732024]] [2.4211926] 0.42894226
7 [[-3.2360206]] [-0.5665953] 0.20374562
8 [[-3.259318]] [6.777541] 0.47161716
9 [[-4.617487]] [5.419437] 0.23832989
10 [[-11.996084]] [-1.9581919] 0.38398337
11 [[-11.996084]] [2.7902565] 0.0016734854
12 [[-11.996085]] [2.9479225] 0.001241757
Tensor raw input:  1.0
Tensor inverted ouput:  0.00011759311
Tensor aproximately calculate NOT function
Tensor raw input:  0.0
Tensor inverted ouput:  0.9501653
Tensor aproximately calculate NOT function
